In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
import librosa
import tensorflow as tf
import keras
from keras.layers import Dense,LSTM,GlobalMaxPool1D, Bidirectional 
from keras.models import Sequential
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn import metrics
import pickle
from scipy.io.wavfile import read,write

# os.chdir('My Drive')
os.chdir('drive/My Drive/')

In [0]:
print(tf.config.list_physical_devices('GPU'))

import warnings
warnings.filterwarnings('ignore')

In [0]:
!pwd
os.listdir()
# os.chdir('My Drive')

In [0]:
activ_dir = 'white_noise'
fs,x = read('white_noise.wav')
file_size = x.shape[0]
segment_time = 3.0
segment_samples = int(segment_time * fs)
no_of_segments = int(file_size/segment_samples)

os.chdir(activ_dir)
for i in range(no_of_segments - 1):
  file_name = '{}_{}.wav'.format(activ_dir, i)
  x_temp = x[i*segment_samples:(i+1)*segment_samples]
  write(file_name, fs, x_temp)


print(fs)
print(x.shape)
print(file_size)
print(segment_samples)
print(no_of_segments)

In [0]:
# KEYWORD_FOLDER1= 'Bachao_Data_Old/'
# KEYWORD_FOLDER2= 'Bachao_Data_Babble_10dB'
# KEYWORD_FOLDER3 = 'Bachao_Data_Natural_10dB'
KEYWORD_FOLDER4 = 'Help_Data_Old'
KEYWORD_FOLDER5 = 'Help_Data_10dB'
KEYWORD_FOLDER6= 'Help_Data_Natural_10dB'


NEGATIVE_FOLDER1 = 'Negative_Data/'
NEGATIVE_FOLDER2 = 'Negative_Data_10dB'
NEGATIVE_FOLDER3 = 'Negative_Data_Natural_10dB'
# NEGATIVE_FOLDER4 = 'Negative_Data/'
# NEGATIVE_FOLDER5 = 'Negative_Data_10dB'
# NEGATIVE_FOLDER6 = 'Negative_Data_Natural_10dB'

#OPPPOSITE_KEYWORD_FOLDER = 'Bachao_Data/'
KEYWORD_FOLDER_TEST = 'Bachao_Data_Test/'
NEGATIVE_FOLDER_TEST = 'Negative_Data_Test_Old/'
#OPPPOSITE_KEYWORD_FOLDER_TEST = 'Bachao_Data_Test/'

In [0]:
direc = os.listdir('Negative_Data')
print(direc)

In [0]:
#os.chdir('My Drive')
INPUT_SHAPE = (376, 40)
dropout = 0.2
var = 0

In [0]:
def create_model():
  print ('Creating model...')
  model = Sequential()
  model.add(Bidirectional(LSTM(units=128, input_shape=INPUT_SHAPE, return_sequences=True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(units=1,activation='sigmoid'))	
  model.compile(loss='binary_crossentropy',
	              optimizer='adam',
	              metrics=['accuracy'])
  return model

In [0]:
def count_files(folder, extension):
	count = 0
	for file in os.listdir(folder):
		if file.endswith(extension):
			file_path = os.path.join(folder, file)
			count += 1
	return count

In [0]:
def load_data_folder(folder, is_keyword):
  num_samples = count_files(folder, '.wav')
  data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
  data_Y = np.zeros((num_samples), dtype=np.float64)

  count = 0
  for file in os.listdir(folder):
    if file.endswith('.wav'):
      file_path = os.path.join(folder, file)
      y, sr = librosa.load(file_path,sr=None)
      mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=128, n_fft=256, n_mfcc=20)
      mfcc_delta = librosa.feature.delta(mfcc)[:10, :]
      mfcc_double_delta = librosa.feature.delta(mfcc, order=2)[:10, :]
      data_X[count, :, :20] = mfcc.T
      data_X[count, :, 20:30] = mfcc_delta.T
      data_X[count, :, 30:] = mfcc_double_delta.T
      data_Y[count] = int(is_keyword)
      count += 1
      if count%50==0:
        print(count)
  return data_X, data_Y

In [0]:
def load_data(folders):
	num_samples = sum([count_files(folder, '.wav') for folder, is_keyword in folders])
	data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
	data_Y = np.zeros((num_samples), dtype=np.float64)
	count = 0
	for folder, is_keyword in folders:
		num_samples_folder = count_files(folder, '.wav')
		data_X[count:count+num_samples_folder, :, :], data_Y[count:count+num_samples_folder] = (
			load_data_folder(folder, is_keyword))
		count += num_samples_folder
	return shuffle(data_X, data_Y, random_state=0)


In [0]:
def load_train_data():
  #folders = [(NEGATIVE_FOLDER_TRAIN_1, False), (NEGATIVE_FOLDER_TRAIN_2, False), (NEGATIVE_FOLDER_TRAIN_3, False)]
  folders = [(KEYWORD_FOLDER4, True), (KEYWORD_FOLDER5, True), (NEGATIVE_FOLDER1, False), (NEGATIVE_FOLDER2, False), (NEGATIVE_FOLDER3, False), (NEGATIVE_FOLDER4, False)]
  #folders = [(KEYWORD_FOLDER1, True), (KEYWORD_FOLDER2, True), (KEYWORD_FOLDER3, True), (KEYWORD_FOLDER4, True), (KEYWORD_FOLDER5, True), (KEYWORD_FOLDER6, True), (KEYWORD_FOLDER_TEST, True), (NEGATIVE_FOLDER1, False), (NEGATIVE_FOLDER2, False), (NEGATIVE_FOLDER3, False), (NEGATIVE_FOLDER4, False), (NEGATIVE_FOLDER5, False), (NEGATIVE_FOLDER6, False), (NEGATIVE_FOLDER_TEST, False)]
  return load_data(folders)

In [0]:
def load_test_data():
  #folders = [(NEGATIVE_FOLDER_TEST, True)]
  folders = [(KEYWORD_FOLDER_TEST, True), (NEGATIVE_FOLDER_TEST, False), (KEYWORD_FOLDER6, True), (NEGATIVE_FOLDER5, False), (NEGATIVE_FOLDER6, False)] 
  return load_data(folders)

In [0]:
%cd My Drive

[Errno 2] No such file or directory: 'My Drive'
/content


In [0]:
#Uncomment these lines to regenerate the MFCC matrix
train_X0, train_Y0 = load_train_data()
print("Train data extracted")

test_X, test_Y = load_test_data()
print("Test data extracted")


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
50
100
150
200
250
300
350
400
450
500
550
600
650
50
100
150
200


In [0]:
import pickle
with open("bachao_help_train_x_noiseless.pickle", "wb") as f:
  pickle.dump(train_X0, f)
print("Test set features done")
with open("bachao_help_train_y_noiseless.pickle", "wb") as f:
  pickle.dump(train_Y0, f)
print("Test set labels done")

'''
with open("train_x2_prime.pickle", "wb") as f:
  pickle.dump(train_X2, f)
print("Train set features done")
with open("train_y2_prime.pickle", "wb") as f:
  pickle.dump(train_Y2, f)
'''
'''
with open("train_x_extra_noise.pickle", "wb") as f:
  pickle.dump(train_X3, f)
print("Train set features done")
with open("train_y_extra_noise.pickle", "wb") as f:
  pickle.dump(train_Y3, f)
'''
print("Train labels done")
# os.listdir('./')

Test set features done
Test set labels done
Train labels done


In [0]:
%ls

 babble/                                help_model110dB.h5
 Babble.wav                             help_model2_2.h5
 Bachao_Data_Babble_10dB/               machinery/
 Bachao_Data_Natural_10dB/              Machinery.wav
 Bachao_Data_Old/                       model_plot.png
 Bachao_Data_Test/                      natural_white_noise.wav
 bachao_help_train_x_10dB.pickle        Negative_Data/
 bachao_help_train_x_noiseless.pickle   Negative_Data_10dB/
 bachao_help_train_y_10dB.pickle        Negative_Data_20dB/
 bachao_help_train_y_noiseless.pickle   Negative_Data_Natural_10dB/
 bachao_model_10dB.h5                   Negative_Data_Test_Old/
 bachao_model_noiseless.h5              test_y_extra_noise.pickle
 bachao_train_x_10dB.pickle             tf_bachao_model_10dB.pb
 bachao_train_x_noiseless.pickle        tf_bachao_model_noiseless.pb
 bachao_train_y_10dB.pickle             tf_help_bachao_model_10dB.pb
 bachao_train_y_noiseless.pickle        tf_help_bachao_model_noiseless.pb
'Colab Note

In [0]:
import pickle
'''
with open('train_x1.pickle', 'rb') as f:
  train_X1 = pickle.load(f)
with open('train_y1.pickle', 'rb') as f:
  train_Y1 = pickle.load(f)
  
with open('train_x2_prime.pickle', 'rb') as f:
  train_X2 = pickle.load(f)
with open('train_y2_prime.pickle', 'rb') as f:
  train_Y2 = pickle.load(f)

with open('test_y3.pickle', 'rb') as f:
  test_Y = pickle.load(f)
with open('test_x3.pickle', 'rb') as f:
  test_X = pickle.load(f)
'''
with open('help_data_total_hari_train_y.pickle', 'rb') as f:
  train_Y2 = pickle.load(f)
with open('help_data_total_hari_train_x.pickle', 'rb') as f:
  train_X2 = pickle.load(f)

with open('help_data_total_hari_test_y.pickle', 'rb') as f:
  test_Y2 = pickle.load(f)
with open('help_data_total_hari_test_x.pickle', 'rb') as f:
  test_X2 = pickle.load(f)

In [0]:
train_X2.shape

(6486, 376, 40)

In [0]:
train_X = np.concatenate((train_X1, train_X2))
train_Y = np.concatenate((train_Y1, train_Y2))

In [0]:
randomize = np.arange(len(train_Y2))
np.random.shuffle(randomize)
train_X_shuffle = train_X2[randomize]
train_Y_shuffle = train_Y2[randomize]

In [0]:
# print(test_X.shape)
# print(test_Y.shape)
print(train_X_shuffle.shape)
print(train_Y_shuffle.shape)

(6486, 376, 40)
(6486,)


In [0]:
import sklearn
train_X_cv, test_X_cv, train_Y_cv, test_Y_cv = sklearn.model_selection.train_test_split(train_X_shuffle, test_X2, train_Y_shuffle, test_Y2)

ValueError: ignored

In [0]:
import sklearn
train_X_cv, test_X_cv, train_Y_cv, test_Y_cv = sklearn.model_selection.train_test_split(train_X0, train_Y0, test_size=0.2)

In [0]:
train_X_cv.shape

(5188, 376, 40)

In [0]:
model = create_model()
model.fit(train_X_cv, train_Y_cv, epochs=5, validation_split=0.2)

Creating model...
Train on 4150 samples, validate on 1038 samples
Epoch 1/5
4150/4150 [==============================] - 142s 34ms/step - loss: 0.4119 - accuracy: 0.8340 - val_loss: 0.2527 - val_accuracy: 0.9220
Epoch 2/5
4150/4150 [==============================] - 139s 33ms/step - loss: 0.1897 - accuracy: 0.9487 - val_loss: 0.1412 - val_accuracy: 0.9672
Epoch 3/5
4150/4150 [==============================] - 137s 33ms/step - loss: 0.1120 - accuracy: 0.9735 - val_loss: 0.1116 - val_accuracy: 0.9653
Epoch 4/5
4150/4150 [==============================] - 135s 33ms/step - loss: 0.0757 - accuracy: 0.9858 - val_loss: 0.0703 - val_accuracy: 0.9875
Epoch 5/5
4150/4150 [==============================] - 135s 32ms/step - loss: 0.0529 - accuracy: 0.9894 - val_loss: 0.0524 - val_accuracy: 0.9904


In [0]:
#Uncomment to save the retrained model
model.save('help_bachao_model_noiseless.h5')

In [0]:
model = keras.models.load_model('help_model_10dB.h5')

In [0]:
predictions = model.predict(test_X2)

In [0]:
print(predictions.shape)

(100, 1)


In [0]:
y_pred = (predictions>0.5)

In [0]:
matrix = metrics.confusion_matrix(test_Y2, y_pred)

In [0]:
matrix

array([[50,  1],
       [44,  5]])